In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [4]:
code="""
#include <stdio.h>
#include <stdlib.h>

#define N 1000000

__global__ void add(int *a, int *b, int *c) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < N) {
        c[tid] = a[tid] + b[tid];
    }
}

int main() {
    int *a, *b, *c;
    int *d_a, *d_b, *d_c;
    int size = N * sizeof(int);

    // Allocate memory on host
    a = (int*)malloc(size);
    b = (int*)malloc(size);
    c = (int*)malloc(size);

    // Initialize arrays
    for (int i = 0; i < N; i++) {
        a[i] = i;
        b[i] = i * 2;
    }

    // Allocate memory on device
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Copy data from host to device
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    // Launch kernel with 1 million threads
    add<<<(N + 255) / 256, 256>>>(d_a, d_b, d_c);

    // Copy result from device to host
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    // Print first and last elements of result
    printf("c[0]=%d, c[%d] = %d",c[0],N-1,c[N-1]);

    // Free memory
    free(a);
    free(b);
    free(c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}
"""

In [8]:
text_file = open('assign4.cu','w')
text_file.write(code)
text_file.close()

In [9]:
!nvcc assign4.cu

In [10]:
!./a.out

c[0]=0, c[999999] = 2999997

In [12]:
!nvprof ./a.out

==2629== NVPROF is profiling process 2629, command: ./a.out
==2629== c[0]=0, c[999999] = 2999997Profiling application: ./a.out
==2629== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   50.80%  1.5404ms         1  1.5404ms  1.5404ms  1.5404ms  [CUDA memcpy DtoH]
                   47.65%  1.4449ms         2  722.45us  679.76us  765.13us  [CUDA memcpy HtoD]
                    1.56%  47.263us         1  47.263us  47.263us  47.263us  add(int*, int*, int*)
      API calls:   97.09%  207.09ms         3  69.031ms  95.429us  206.90ms  cudaMalloc
                    2.14%  4.5651ms         3  1.5217ms  896.36us  2.7277ms  cudaMemcpy
                    0.44%  931.97us         1  931.97us  931.97us  931.97us  cuDeviceGetPCIBusId
                    0.25%  526.02us         3  175.34us  115.38us  206.47us  cudaFree
                    0.05%  115.58us       101  1.1440us     126ns  47.206us  cuDeviceGetAttribute
                